In [3]:
from tensorflow.keras.applications import MobileNetV2, Xception, EfficientNetB0
from tensorflow.keras.layers import Concatenate, Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

dic = {0: 'Buras', 1: 'Dangkot', 2: 'Gogos', 3: 'Kapurung', 4: 'Sokko', 5: 'Sop Konro'} 
label_to_class = {v: k for k, v in dic.items()} 

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img 

# Fungsi ensemble predict menggabungkan 3 model
def ensemble_predict(img_path, mobilenetv2_model, xception_model, efficientnetb0_model):
    img = preprocess_image(img_path)
    mobilenetv2_pred = mobilenetv2_model.predict(img)
    xception_pred = xception_model.predict(img)
    efficientnetb0_pred = efficientnetb0_model.predict(img)
    final_pred = (mobilenetv2_pred + xception_pred + efficientnetb0_pred) / 3.0  # Rata-rata probabilitas
    return np.argmax(final_pred, axis=-1) 

# Input model
mobilenetv2_model = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\model\mobilenetv2_Percobaan 4(2).h5")
xception_model = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Percobaan4_Xception.h5") 
efficientnetb0_model = load_model(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\Percobaan3(1)_EfficientNetB0.h5")

# Fungsi untuk menyimpan model ensemble dalam format H5
def save_ensemble_model(mobilenetv2_model, xception_model, efficientnetb0_model):
    mobilenetv2_input = mobilenetv2_model.input
    xception_input = xception_model.input
    efficientnetb0_input = efficientnetb0_model.input

    # Ambil output dari layer terakhir dari masing-masing model
    mobilenetv2_output = mobilenetv2_model.output
    xception_output = xception_model.output
    efficientnetb0_output = efficientnetb0_model.output

    # Gabungkan output dari ketiga model
    merged_output = Concatenate()([mobilenetv2_output, xception_output, efficientnetb0_output])

    # Tambahkan lapisan Dense untuk menghasilkan prediksi akhir
    final_output = Dense(6, activation='softmax')(merged_output)

    # Buat model ensemble
    ensemble_model = Model(inputs=[mobilenetv2_input, xception_input, efficientnetb0_input], outputs=final_output)

    # Simpan model ensemble dalam format H5
    ensemble_model.save(r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Codingan\model_ensemble.h5")

save_ensemble_model(mobilenetv2_model, xception_model, efficientnetb0_model)

# Evaluasi model
def evaluate_model(test_images, test_labels):
    true_labels = []
    pred_labels = []
    
    for img_path, true_label in zip(test_images, test_labels):
        pred_label = ensemble_predict(img_path, mobilenetv2_model, xception_model, efficientnetb0_model)[0]
        true_labels.append(true_label)
        pred_labels.append(pred_label)
    
    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, average='weighted')
    recall = recall_score(true_labels, pred_labels, average='weighted')
    f1 = f1_score(true_labels, pred_labels, average='weighted')
    
    print("Accuracy: ", accuracy)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1 Score: ", f1)
    print("Classification Report: \n", classification_report(true_labels, pred_labels, target_names=list(dic.values())))

test_images = [
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Buras\BURAS_3.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Dangkot\DANGKOT_7.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Gogos\GOGOS_22.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Kapurung\KAPURUNG_27.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Sokko\SOKKO_6.jpg",
    r"D:\Kuliah Semester 6\Jurnal\Materi\CNN\Dataset\Sop Konro\SOP KONRO_188.jpg"
]
test_labels = [
    0,
    1,
    2,
    3,
    4,
    5
]

evaluate_model(test_images, test_labels)


ValueError: The name "input_layer" is used 3 times in the model. All operation names should be unique.